# 1. 데이터 로드

In [4]:
import pandas as pd
selloutData = pd.read_csv("../Python_ST_EX/dataset/kopo_channel_seasonality_new.csv")
selloutData.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0


# 2. 데이터 정제하기

## 데이터가 음수인 값 양수로 변경

In [5]:
import numpy as np
selloutData["NEW_QTY"] = np.where(selloutData["QTY"] < 0, 0, selloutData["QTY"])
selloutData[selloutData.QTY < 0].head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY
65,A02,PRODUCT16,201403,-1.0,0.0


## 53주차 제거

In [6]:
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
NEW_QTY     float64
dtype: object

In [7]:
# YEARWEEK가 Int형일 경우 처리
refinedData = selloutData[selloutData.YEARWEEK % 100 < 53]
refinedData[refinedData.YEARWEEK % 100 > 52]

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY


## 연주차 정보 분리

In [8]:
# "//" 연산자는 결과를 정수형으로 산출 "/" 연산자는 실수형으로 산출
refinedData["YEAR"] = refinedData["YEARWEEK"] // 100
refinedData["WEEK"] = refinedData["YEARWEEK"] % 100
refinedData = refinedData[["REGIONID", "PRODUCT", "YEARWEEK", "YEAR", "WEEK", "QTY", "NEW_QTY"]]
refinedData.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,REGIONID,PRODUCT,YEARWEEK,YEAR,WEEK,QTY,NEW_QTY
0,A60,PRODUCT4,201402,2014,2,71.0,71.0
1,A60,PRODUCT59,201402,2014,2,22275.0,22275.0
2,A60,PRODUCT34,201402,2014,2,4463.0,4463.0
3,A60,PRODUCT47,201402,2014,2,0.0,0.0
4,A60,PRODUCT56,201402,2014,2,23.0,23.0


In [9]:
sortKey = ["REGIONID", "PRODUCT", "YEARWEEK"]
# drop이 True면 기존의 인덱스를 없앤다는 뜻
sortedData = refinedData.sort_values(sortKey).reset_index(drop=True)
sortedData.head(20)

,REGIONID,PRODUCT,YEARWEEK,YEAR,WEEK,QTY,NEW_QTY
0,A00,PRODUCT34,201401,2014,1,661.0,661.0
1,A00,PRODUCT34,201402,2014,2,679.0,679.0
2,A00,PRODUCT34,201403,2014,3,578.0,578.0
3,A00,PRODUCT34,201404,2014,4,532.0,532.0
4,A00,PRODUCT34,201405,2014,5,516.0,516.0
5,A00,PRODUCT34,201406,2014,6,423.0,423.0
6,A00,PRODUCT34,201407,2014,7,407.0,407.0
7,A00,PRODUCT34,201408,2014,8,364.0,364.0
8,A00,PRODUCT34,201409,2014,9,470.0,470.0
9,A00,PRODUCT34,201410,2014,10,538.0,538.0


# 3. 이동평균 구하기

In [10]:
step1Data = sortedData
order = 5
suborder = order // 2
suborder

2

In [11]:
step1Data["MA"] = step1Data.NEW_QTY.rolling(window = 15, center = True, min_periods = 8).mean()
step1Data["MSTD"] = step1Data.MA.rolling(window = 9, center = True, min_periods = 5).std(ddof = 1)
step1Data.head()

,REGIONID,PRODUCT,YEARWEEK,YEAR,WEEK,QTY,NEW_QTY,MA,MSTD
0,A00,PRODUCT34,201401,2014,1,661.0,661.0,520.000000,12.002994
1,A00,PRODUCT34,201402,2014,2,679.0,679.0,514.444444,15.933923
2,A00,PRODUCT34,201403,2014,3,578.0,578.0,516.800000,19.930825
3,A00,PRODUCT34,201404,2014,4,532.0,532.0,503.363636,23.784855
4,A00,PRODUCT34,201405,2014,5,516.0,516.0,490.750000,30.428905


# 4. 그룹화

In [12]:
groupKey = ["REGIONID","PRODUCT","YEAR"]
groupData = step1Data.groupby(groupKey, as_index = False)[["NEW_QTY"]].mean()
groupData.columns = ["REGIONID", "PRODUCT","YEAR","GROUP"]
groupData.head()

,REGIONID,PRODUCT,YEAR,GROUP
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538


# 5. 데이터 합치기

In [13]:
mergedData = pd.merge(step1Data, groupData, on = groupKey, how = "left")
mergedData.tail()

,REGIONID,PRODUCT,YEARWEEK,YEAR,WEEK,QTY,NEW_QTY,MA,MSTD,GROUP
123859,A77,PRODUCT12,201648,2016,48,4152.0,4152.0,4839.500000,260.550638,4837.153846
123860,A77,PRODUCT12,201649,2016,49,5086.0,5086.0,4914.909091,274.767625,4837.153846
123861,A77,PRODUCT12,201650,2016,50,5846.0,5846.0,4977.200000,229.386294,4837.153846
123862,A77,PRODUCT12,201651,2016,51,4933.0,4933.0,4890.222222,91.131622,4837.153846
123863,A77,PRODUCT12,201652,2016,52,7551.0,7551.0,4987.875000,61.751302,4837.153846


# 6. 그룹화 함수 만들기

In [14]:
groupKey = ["REGIONID","PRODUCT"]
groupDataSet = mergedData.groupby(groupKey)
groupDataSet

In [15]:
len(list(groupDataSet.groups))

794

In [16]:
oneGroup = groupDataSet.get_group(list(groupDataSet.groups)[0])
oneGroup

,REGIONID,PRODUCT,YEARWEEK,YEAR,WEEK,QTY,NEW_QTY,MA,MSTD,GROUP
0,A00,PRODUCT34,201401,2014,1,661.0,661.0,520.000000,12.002994,275.961538
1,A00,PRODUCT34,201402,2014,2,679.0,679.0,514.444444,15.933923,275.961538
2,A00,PRODUCT34,201403,2014,3,578.0,578.0,516.800000,19.930825,275.961538
3,A00,PRODUCT34,201404,2014,4,532.0,532.0,503.363636,23.784855,275.961538
4,A00,PRODUCT34,201405,2014,5,516.0,516.0,490.750000,30.428905,275.961538
5,A00,PRODUCT34,201406,2014,6,423.0,423.0,480.230769,38.666084,275.961538
6,A00,PRODUCT34,201407,2014,7,407.0,407.0,468.214286,47.192218,275.961538
7,A00,PRODUCT34,201408,2014,8,364.0,364.0,456.666667,53.509329,275.961538
8,A00,PRODUCT34,201409,2014,9,470.0,470.0,431.533333,59.591171,275.961538
9,A00,PRODUCT34,201410,2014,10,538.0,538.0,402.066667,64.038426,275.961538


In [17]:
def rolling_function(dataFrame, groupKey, rollingColumnName, window = 15,aggFunc = 0):
    groupDataSet = dataFrame.groupby(groupKey)
    groupDataSetList = list(groupDataSet.groups)
    rollingList = []
    for i in range(0, len(groupDataSetList)):
        rollingValue = groupDataSet.get_group(groupDataSetList[i])
        if (aggFunc == 0):
            rollingList += list(rollingValue[rollingColumnName].rolling(window = window, center = True, min_periods = 0).mean())
        else:
            rollingList += list(rollingValue[rollingColumnName].rolling(window = window, center = True, min_periods = 0).std(ddof = 1))
    return rollingList

In [18]:
list1 = rolling_function(dataFrame = mergedData, groupKey = ["REGIONID", "PRODUCT"], rollingColumnName = "NEW_QTY")

In [19]:
list1 

[520.0,
 514.4444444444445,
 516.8,
 503.3636363636364,
 490.75,
 480.2307692307692,
 468.2142857142857,
 456.6666666666667,
 431.53333333333336,
 402.06666666666666,
 375.53333333333336,
 350.8,
 325.73333333333335,
 305.26666666666665,
 284.6666666666667,
 265.93333333333334,
 239.33333333333334,
 207.8,
 187.2,
 169.2,
 155.66666666666666,
 153.13333333333333,
 168.93333333333334,
 179.93333333333334,
 193.0,
 205.26666666666668,
 217.6,
 229.06666666666666,
 237.6,
 246.2,
 253.4,
 261.4,
 269.3333333333333,
 275.0,
 282.73333333333335,
 285.6666666666667,
 276.8666666666667,
 251.8,
 235.13333333333333,
 222.93333333333334,
 209.46666666666667,
 194.33333333333334,
 187.86666666666667,
 179.73333333333332,
 172.73333333333332,
 170.06666666666666,
 170.06666666666666,
 164.93333333333334,
 165.53333333333333,
 156.93333333333334,
 148.33333333333334,
 142.86666666666667,
 135.93333333333334,
 125.46666666666667,
 112.73333333333333,
 107.4,
 103.13333333333334,
 90.53333333333333,

In [20]:
def groupRolling(oneGroup):
    indexGroupData = oneGroup.reset_index(drop = True)
    indexGroupData["MA"] = indexGroupData["QTY"].rolling(window = 15, center = True, min_periods = 0).mean()
    return indexGroupData
finalResult = mergedData.groupby(groupKey).apply(groupRolling).reset_index(drop = True)

In [21]:
finalResult

,REGIONID,PRODUCT,YEARWEEK,YEAR,WEEK,QTY,NEW_QTY,MA,MSTD,GROUP
0,A00,PRODUCT34,201401,2014,1,661.0,661.0,520.000000,12.002994,275.961538
1,A00,PRODUCT34,201402,2014,2,679.0,679.0,514.444444,15.933923,275.961538
2,A00,PRODUCT34,201403,2014,3,578.0,578.0,516.800000,19.930825,275.961538
3,A00,PRODUCT34,201404,2014,4,532.0,532.0,503.363636,23.784855,275.961538
4,A00,PRODUCT34,201405,2014,5,516.0,516.0,490.750000,30.428905,275.961538
5,A00,PRODUCT34,201406,2014,6,423.0,423.0,480.230769,38.666084,275.961538
6,A00,PRODUCT34,201407,2014,7,407.0,407.0,468.214286,47.192218,275.961538
7,A00,PRODUCT34,201408,2014,8,364.0,364.0,456.666667,53.509329,275.961538
8,A00,PRODUCT34,201409,2014,9,470.0,470.0,431.533333,59.591171,275.961538
9,A00,PRODUCT34,201410,2014,10,538.0,538.0,402.066667,64.038426,275.961538


In [22]:
# 마지막 4주차 평균
def rolling_function(dataFrame, groupKey, rollingColumnName):
    groupDataSet = dataFrame.groupby(groupKey)
    groupDataSetList = list(groupDataSet.groups)
    rollingList = []
    for i in range(0, len(groupDataSetList)):
        rollingValue = groupDataSet.get_group(groupDataSetList[i])
        rollingList += [sum(rollingValue[rollingColumnName][-4:]) / 4]
    return rollingList

In [27]:
a = rolling_function(dataFrame = mergedData, groupKey = ["REGIONID", "PRODUCT"], rollingColumnName = "NEW_QTY")
b = rolling_function(dataFrame = sortedData, groupKey = ["REGIONID", "PRODUCT"], rollingColumnName = "NEW_QTY")

In [29]:
a == b

True